In [ ]:
! pip install requests beautifulsoup4 certifi pyopenssl urllib3 html5lib pandas tqdm sqlalchemy psycopg2

In [ ]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
from pandas import read_csv
from datetime import datetime
from tqdm import tqdm
from sqlalchemy import create_engine, URL
import json
from pathlib import Path


In [ ]:
categort_list = [
    ("category/game-news", 1668),
    ("category/articles", 119),
    ("tag/รีวิวเกม", 112),
    ("tag/ข่าวไอที", 15),
    ("tag/เกมออนไลน์", 1471),
    ("tag/เกมมือถือน่าเล่น", 12),
    ("tag/เกมมือถือ", 870),
    ("tag/เกมเปิดใหม่", 67),
    ("tag/เกมออนไลน์-2022", 189),
    ("tag/เกม-pc", 78),
]


In [ ]:
article = []
for cat, max in categort_list:
    for i in tqdm(range(1, max + 1), desc=cat):
        data = requests.get(f"https://www.gamemonday.com/{cat}/page/{i}")
        soup = BeautifulSoup(data.content, "html5lib")
        for ar in soup.find_all("article"):
            href = ar.figure.a["href"]
            if "articles" in href:
                article.append(href)


In [ ]:
with open("game.txt", "w") as f:
    for a in set(article):
        f.write(f"{a}\n")


---------

In [ ]:
with open("game.txt", "r") as f:
    article = list(map(str.strip, f.readlines()))


In [ ]:
db_data = {}
filename = "game_{i}.csv"

for i, url in tqdm(enumerate(article), total=len(article)):
    if i % 100 == 0 and len(db_data) >= 100:
        df = DataFrame(db_data).transpose()
        df.to_csv(filename.format(i))
        db_data = {}

    data = requests.get(url)
    soup = BeautifulSoup(data.content, "html5lib")
    try:
        paragraph = []
        for p in soup.find("div", {"class": "entry-content mb-4"})("p"):
            text = p.text.replace("\xa0", " ")
            if len(text) < 150:
                continue
            paragraph.append(text)

        db_data[url] = {
            "url": url,
            "category": "Game",
            "title": soup.title.string,
            "content": None,
            "paragraphs": json.dumps(paragraph, ensure_ascii=False),
            "raw_html": data.text,
            "raw_json": None,
            "remark": None,
            "created_at": datetime.now(),
        }
    except Exception as e:
        print(f"Exception on {url} by {e.args}")

df = DataFrame(db_data).transpose()
df.to_csv(filename.format(i+1))


---

In [ ]:
engine = create_engine()


In [ ]:
list_path = list(Path(".").glob("game_*.csv"))
for path in tqdm(list_path):
    df = read_csv(path)
    df.drop(columns=["Unnamed: 0"], inplace=True)
    df.to_sql("article_raw_data", engine, index=False, if_exists="append", method="multi")
